In [3]:

import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
date_picker = widgets.DatePicker(
        description='Select a Date',
        disabled=False
    )
date_picker

DatePicker(value=None, description='Select a Date')

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt

def plot_ef_with_date_input(n_points, er, cov, er2, cov2, returns_df1, returns_df2):
    """
    Plots two multi-asset efficient frontiers on the same graph with a date input widget
    """
    date_picker = widgets.DatePicker(
        description='Select a Date',
        disabled=False
    )

    output_date = widgets.Output()

    def plot_efficient_frontiers(selected_date):
        with output_date:
            # Filter dataframes based on the date the user selected
            try:
                returns_data1 = returns_df1[returns_df1['Date'] == selected_date]
                returns_data2 = returns_df2[returns_df2['Date'] == selected_date]
            except KeyError:
                print("Column names in returns_df1:", returns_df1.columns)
                print("Column names in returns_df2:", returns_df2.columns)
                print("Error: 'Date' column not found in the DataFrame.")
                return

            weights = optimal_weights(n_points, er, cov)
            rets = [portfolio_return(w, er) for w in weights]
            vols = [portfolio_vol(w, cov) for w in weights]
            ef1 = pd.DataFrame({"Returns": rets, "Volatility": vols})

            weights2 = optimal_weights(n_points, er2, cov2)
            rets2 = [portfolio_return(w, er2) for w in weights2]
            vols2 = [portfolio_vol(w, cov2) for w in weights2]
            ef2 = pd.DataFrame({"Returns": rets2, "Volatility": vols2})

            # Plotting both efficient frontiers on the same graph
            ax = ef1.plot.line(x="Volatility", y="Returns", style='.-', label='Industry portfolio without cryptocurrencies')
            ef2.plot.line(x="Volatility", y="Returns", style='.-', ax=ax, label='Industry portfolio with cryptocurrencies')

            ax.set_xlim(left=0.0114, right=0.016)
            ax.set_ylim(bottom=0.04, top=0.14)

            # Customize the plot as needed
            plt.title("Efficient Frontier Comparison")
            plt.xlabel("Volatility")
            plt.ylabel("Returns")
            plt.legend()
            plt.grid(True)
            plt.show()

            return ax

    initial_date = date_picker.value
    ax = plot_efficient_frontiers(initial_date)

    def on_date_change(change):
        with output_date:
            # Clear previous output
            output_date.clear_output(wait=True)
            # Display the selected date
            print(f"Selected Date: {change['new']}")
            # Update the plot based on the selected date
            ax = plot_efficient_frontiers(change['new'])

    date_picker.observe(on_date_change, names='value')

    # Display the widgets
    display(date_picker)
    display(output_date)

industry_crypto_combined_plots = plot_ef_with_date_input(200, er[industry], cov.loc[industry, industry], er[industry_and_crypto], cov.loc[industry_and_crypto, industry_and_crypto], test_industry, test_crypto)